# Part 2: OpenAI API Function Calling

## Pizza Buchungssystem

Task: Der digitale Kellner soll Bestellungen in einer Bestellapp registrieren können. Die Bestellapp stellt dazu eine Funktion "add_item_to_order" bereit. Diese bekommt als Parameter die Id eines Menueintrags übergeben und fügt den Menüeintrag dann zu einer Liste hinzu.  

ChatGPT unterstützt sogenanntes Function Calling. Dabei erhält das Modell eine Beschreibung verfügbarer Funktionen inklusive ihrer Argumente. Basierend auf dem Nutzereingaben kann ChatGPT passende Funktionsaufrufe vorschlagen, die anschließend lokal ausgeführt werden können.

Zwar lässt sich eine vergleichbare Funktionalität grundsätzlich auch über strukturierte Ausgaben realisieren, jedoch stellt Function Calling eine einfachere und standardisierte Methode dafür dar.

**Aufgabe:** Erweitere die Funktionalität um eine Funktion "remove_item_from_order", welche die id eines bestelltem Items übergeben bekommt und dieses aus der Liste löscht. 

## Setup

In [ ]:
# Install required packages
%pip install openai sentence-transformers matplotlib scikit-learn umap-learn plotly faiss-cpu numpy tabulate pandas

# Import necessary libraries
from openai import OpenAI
import json
import faiss
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from typing import List, Dict
import io

# Download Menu module und data
import urllib.request
import os.path
MENU_URL = "https://raw.githubusercontent.com/jank-bcxp/bcxp_weekend2025_HandsOn_AI/refs/heads/main/menu.py"
urllib.request.urlretrieve(MENU_URL, os.path.basename(MENU_URL))

from menu import MENU

# Initialize OpenAI client
openai_client = OpenAI(api_key= "")

## Function & Tool Definitions

In [ ]:
from typing import List, Dict

# Bestellung (als globale Liste)
bestellung: List[Dict] = []

def add_item_to_order(item_id: str):
    item = next((entry for entry in MENU if entry["id"] == item_id), None)
    if item:
        bestellung.append(item)
        return f"{item['name']} wurde zur Bestellung hinzugefügt."
    return f"Item mit ID {item_id} nicht gefunden."

In [ ]:
tools = [
    {
        "type": "function",
        "name": "add_item_to_order",
        "description": "Fügt ein Item der Bestellung hinzu.",
        "parameters": {
            "type": "object",
            "properties": {
                "item_id": {
                    "type": "string",
                    "description": "Die ID des hinzuzufügenden Items.",
                }
            },
            "required": ["item_id"],
        },
    },
]

## Function Calling

In [ ]:
import json
from openai import OpenAI
from hack_helpers import OPENAI_API_KEY

client = OpenAI(api_key=OPENAI_API_KEY)

def chat_with_waiter(user_input: str):
    print(f"👤 Input: {user_input}")
    # Erstellung der Eingabenachrichten
    input_messages = [
        {
            "role": "developer",
            "content": "Du bist ein digitaler Kellner. Nimm Bestellungen auf und beantworte Fragen zum Menü.",
        },
        {
            "role": "developer",
            "content": f"Speisekarte:\n{json.dumps(MENU, ensure_ascii=False)}",
        },
        {
            "role": "user",
            "content": user_input,
        },

    ]

    # API-Aufruf mit Function Calling
    response = client.responses.create(
        model="gpt-4o", input=input_messages, tools=tools, tool_choice="auto"
    )

    for tool_call in response.output:
        if tool_call.type != "function_call":
            print(f"🤖 Antwort: {response.output_text}")
            continue

        name = tool_call.name
        args = json.loads(tool_call.arguments)

        if name == "add_item_to_order":
            result = add_item_to_order(args["item_id"])
        print(f"🔧 Function Call: {name}({args}) ➜ {result}")

    # Anzeige der aktuellen Bestellung
    print("📦 Aktuelle Bestellung:")
    for item in bestellung:
        print(f"- {item['name']} ({item['preis']} €)")
    print('-' * 40+'\n')

In [ ]:
chat_with_waiter("Wie geht es dir? ")
chat_with_waiter("Ich möchte gerne eine Pizza Diavola und eine Cola bestellen.")
chat_with_waiter("Noch eine Cola bitte")
chat_with_waiter("Bitte entferne die Cola wieder.")